In [1]:
import spacy
import tqdm
from spacy.training import Example

In [2]:
import json
labeled_data = []
with open("tagged_data.json", "r") as read_file:
    data = json.load(read_file)

In [3]:
train_data = []
for i in range(954):
    
    string = data[0]['paragraphs'][i]['raw']
    entities = []
    for item in data[0]['paragraphs'][i]['entities']:
        ety = (item[0],item[1],item[2])
        entities.append(ety)
        
    d = {'entities':entities}
    b = (string,d)
    train_data.append(b)

In [6]:
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

for text, annot in tqdm(train_data): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("train.spacy")

100%|██████████| 954/954 [00:00<00:00, 1314.52it/s]


In [4]:
nlp1 = spacy.load(R"output/model-best")

In [5]:
pred_list = []
for item,_ in train_data:
    pred_list.append(item)

In [19]:
doc = nlp1(pred_list[20]) # input sample text

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

In [9]:
entities=[(i, i.label_, i.label) for i in doc.ents]

In [10]:
entities

[]

## Evaluation

In [235]:
from spacy.scorer import Scorer

In [236]:
examples = []
for text, annots in train_data:
    examples.append(Example.from_dict(nlp1(text), annots))

In [237]:
scorer = Scorer(nlp1)

In [238]:
scorer.score(examples)

{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'ents_p': 0.9995854063018242,
 'ents_r': 1.0,
 'ents_f': 0.9997926601700187,
 'ents_per_type': {'date': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'medicine': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'payment': {'p': 0.996694214876033, 'r': 1.0, 'f': 0.9983443708609271},
  'clinic_name': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'address': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'contact': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'website': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'receipt_number': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'name': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'email': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'service': {'p': 1.0, 'r': 1.0, 'f': 1.0}}}